In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re

src_dir = r'D:\Ravi_Data_Engineer\02_Target\01_raw'
tgt_dir = r'D:\Ravi_Data_Engineer\02_Target\02_cleansed'
os.makedirs(tgt_dir, exist_ok=True)

def clean_datefields(df, output_format='%Y-%m-%d'):
    for col in df.columns:
        if "date" in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
            df[col] = df[col].dt.strftime(output_format)
    return df

def clean_products(df, group_col='Category'):
    if group_col not in df.columns:
        print(f"'{group_col}' column not found. Skipping product-specific cleaning.")
        return df
    numeric_cols = df.select_dtypes(include='number').columns
    for col in numeric_cols:
        df[col] = df.groupby(group_col)[col].transform(lambda x: x.fillna(x.mean()))
        df[col] = df[col].round(2)
    return df

def clean_transactions(df):
    numeric_cols = df.select_dtypes(include='number').columns
    df[numeric_cols] = df[numeric_cols].fillna(0).round(2)
    return df

def normalize_phone(phone):
    digits = re.sub(r'\D', '', str(phone))
    return digits if len(digits) == 10 else '1234567890'

keyword = input("Enter the keyword to search in file names:").lower()

# Delete existing target file if it matches keyword
for f in os.listdir(tgt_dir):
    if keyword in f.lower():
        os.remove(os.path.join(tgt_dir, f))
        print(f"Deleted existing file from target: {f}")

# Identify source files
files_list_src = [f for f in os.listdir(src_dir) if keyword in f.lower()]
if not files_list_src:
    print(f"No files found with keyword '{keyword}' in {src_dir}.")

cleaned_dfs = []

for file in files_list_src:
    file_path = os.path.join(src_dir, file)
    print(f"\nReading: {file_path}")
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip().str.replace(r'\.\d+$', '', regex=True)
        df = df.loc[:, ~df.columns.duplicated()]
        df = df.dropna(how='all')

        string_cols = df.select_dtypes(include='object').columns
        df[string_cols] = df[string_cols].fillna('Unknown')

        numeric_cols = df.select_dtypes(include='number').columns
        df[numeric_cols] = df[numeric_cols].round(2)

        df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

        if 'Phone' in df.columns:
            df['Phone'] = df['Phone'].apply(normalize_phone)

        df = clean_datefields(df)

        if keyword == 'products':
            df = clean_products(df)
        elif keyword == 'transactions':
            df = clean_transactions(df)

        cleaned_dfs.append((file, df))
        print(f" Cleaned: {file} - {df.shape[0]} rows, {df.shape[1]} columns")

    except Exception as e:
        print(f" Failed to read {file}: {e}")
        continue

# Save cleaned files
for file_name, df in cleaned_dfs:
    orig_name, ext = os.path.splitext(file_name)
    new_name = f"cleansed_{orig_name.replace('raw_', '')}{ext}"
    tgt_file = os.path.join(tgt_dir, new_name)
    try:
        df.to_csv(tgt_file, index=False)
        print(f" Saved cleaned data to: {tgt_file}")
    except Exception as e:
        print(f" Failed to save {file_name}: {e}")


Deleted existing file from target: cleansed_retail_transactions.csv

Reading: D:\Ravi_Data_Engineer\02_Target\01_raw\raw_retail_transactions.csv
 Cleaned: raw_retail_transactions.csv - 500000 rows, 8 columns
 Saved cleaned data to: D:\Ravi_Data_Engineer\02_Target\02_cleansed\cleansed_retail_transactions.csv
